In [37]:
import onnxruntime as rt
import uproot
import numpy as np
import tensorflow as tf
import os

In [38]:
columns = [
    "era",
    "higgs1_reco_pt",
    "higgs1_reco_eta",
    "higgs1_reco_phi",
    "higgs1_reco_mass",
    "higgs2_reco_pt",
    "higgs2_reco_eta",
    "higgs2_reco_phi",
    "higgs2_reco_mass",
    "HT",
    "higgs1_DeltaRjj",
    "higgs2_DeltaRjj",
    "minDeltaR_Higgjj",
    "maxDeltaR_Higgjj",
    "higgs1_helicityCosTheta",
    "higgs2_helicityCosTheta",
    "hh_CosThetaStar_CS",
    "hh_vec_mass",
    "hh_vec_pt",
    "hh_vec_eta",
    "hh_vec_DeltaR",
    "hh_vec_DeltaPhi",
    "hh_vec_DeltaEta",
    "higgs1_reco_jet1_pt",
    "higgs1_reco_jet1_eta",
    "higgs1_reco_jet1_phi",
    "higgs1_reco_jet1_mass",
    "higgs1_reco_jet2_pt",
    "higgs1_reco_jet2_eta",
    "higgs1_reco_jet2_phi",
    "higgs1_reco_jet2_mass",
    "higgs2_reco_jet1_pt",
    "higgs2_reco_jet1_eta",
    "higgs2_reco_jet1_phi",
    "higgs2_reco_jet1_mass",
    "higgs2_reco_jet2_pt",
    "higgs2_reco_jet2_eta",
    "higgs2_reco_jet2_phi",
    "higgs2_reco_jet2_mass",
    "add_jet1pt_pt",
    "add_jet1pt_eta",
    "add_jet1pt_phi",
    "add_jet1pt_mass",
    "sigma_over_higgs1_reco_mass",
    "sigma_over_higgs2_reco_mass",
]
main_dir='/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/keras_models_morphing'


In [39]:
input_root=f'/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/JetMET_2022EE_2b_signal_region_to_4b_soumya_january2025.root'
tree=uproot.open(input_root)["tree"]
print(tree.keys())


['era', 'event', 'HT', 'jet_pt', 'jet_pt_raw', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_mass_raw', 'jet_deepjet_probBVsAll', 'jet_pnet_probBVsAll', 'jet_pnet_probTauVsJet', 'jet_pnet_probb', 'jet_pnet_probc', 'jet_pnet_probuds', 'jet_pnet_probtauh', 'jet_pnet_ptcorr', 'jet_pnet_ptnu', 'jet_pnet_ptres', 'jet_pnet_jec_L1L2L3', 'jet_pnet_jec_L2Rel', 'jet_pnet_jec_L2L3Res', 'btag_cat', 'weight', 'is_boosted', 'weight_dnn', 'sigma_higgs1', 'sigma_over_higgs1_reco_mass', 'higgs1_reco_pt', 'higgs1_reco_eta', 'higgs1_reco_phi', 'higgs1_reco_mass', 'sigma_higgs2', 'sigma_over_higgs2_reco_mass', 'higgs2_reco_pt', 'higgs2_reco_eta', 'higgs2_reco_phi', 'higgs2_reco_mass', 'hh_vec_mass', 'hh_vec_pt', 'hh_vec_eta', 'hh_vec_phi', 'hh_vec_DeltaR', 'hh_vec_DeltaPhi', 'hh_vec_DeltaEta', 'hh_vec_ptOmass', 'njet', 'higgs1_reco_jet1_pt', 'higgs1_reco_jet1_eta', 'higgs1_reco_jet1_phi', 'higgs1_reco_jet1_mass', 'higgs1_reco_jet2_pt', 'higgs1_reco_jet2_eta', 'higgs1_reco_jet2_phi', 'higgs1_reco_jet2_mass', 'hig

get the columns from the tree

In [40]:
input_data_dict = tree.arrays(columns, library="np")
print(input_data_dict, type(input_data_dict), len(input_data_dict['era']))


{'era': array([1, 1, 1, ..., 1, 1, 1], dtype=int32), 'higgs1_reco_pt': array([114.53315, 152.08524, 178.58029, ..., 267.7927 , 284.75745,
       204.16527], dtype=float32), 'higgs1_reco_eta': array([-0.5849934 ,  0.05618766, -1.7868915 , ...,  1.5282159 ,
       -0.09514122, -1.9165224 ], dtype=float32), 'higgs1_reco_phi': array([ 0.51116806,  0.9747667 ,  2.7567482 , ..., -1.0621026 ,
        2.113538  ,  1.722217  ], dtype=float32), 'higgs1_reco_mass': array([127.91875 , 104.56954 , 134.2589  , ..., 118.74667 , 138.50037 ,
       118.519966], dtype=float32), 'higgs2_reco_pt': array([ 97.30428 , 104.97775 , 106.02391 , ..., 246.28754 , 190.73148 ,
        59.150402], dtype=float32), 'higgs2_reco_eta': array([-0.9684525 ,  1.3636122 , -0.53399426, ..., -0.13717638,
       -1.3410919 ,  1.5108076 ], dtype=float32), 'higgs2_reco_phi': array([ 1.3662084 , -1.4257993 , -0.10356549, ...,  1.9001577 ,
       -0.87525326, -1.5235088 ], dtype=float32), 'higgs2_reco_mass': array([120.70619, 101

In [62]:
n_events = 10
# get the input data as a numpy array
input_data = np.array([input_data_dict[col][:n_events] for col in columns], dtype=np.float32).T
print(input_data, type(input_data))
weight_dnn = np.array(tree['weight_dnn'].array()[:n_events])
input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
print(input_tensor, type(input_tensor))


[[ 1.00000000e+00  1.14533150e+02 -5.84993422e-01  5.11168063e-01
   1.27918747e+02  9.73042831e+01 -9.68452513e-01  1.36620843e+00
   1.20706192e+02  3.21412842e+02  2.00027561e+00  1.77813196e+00
   5.81731975e-01  3.02608585e+00  1.16788380e-01  5.86703271e-02
   4.05461282e-01  2.67045593e+02  1.92903671e+02 -8.35395277e-01
   9.37088549e-01 -8.55040371e-01  3.83459121e-01  1.23513275e+02
  -2.92204082e-01  8.75747621e-01  1.53098450e+01  4.40478973e+01
  -7.15619206e-01 -1.07920051e+00  4.86812973e+00  8.17293701e+01
  -6.93214834e-01  2.17798185e+00  1.21713457e+01  7.21222916e+01
  -6.28441095e-01  4.01030064e-01  5.63833618e+00 -1.00000000e+01
  -1.00000000e+01 -1.00000000e+01 -1.00000000e+01  1.09902352e-01
   6.81678504e-02]
 [ 1.00000000e+00  1.52085236e+02  5.61876558e-02  9.74766672e-01
   1.04569542e+02  1.04977753e+02  1.36361217e+00 -1.42579925e+00
   1.01880623e+02  3.64698059e+02  1.63543606e+00  1.73896706e+00
   1.63543606e+00  2.98150039e+00  1.88756943e-01  9.6107

In [63]:
session_combine = rt.InferenceSession(
    f"{main_dir}/average_model_from_keras.onnx",
    providers=rt.get_available_providers()
)
# print the input/putput name and shape
input_name=[input.name for input in session_combine.get_inputs()]
output_name=[output.name for output in session_combine.get_outputs()]
print("Inputs name:", input_name)
print("Outputs name:", output_name)

input_shape=[input.shape for input in session_combine.get_inputs()]
output_shape=[output.shape for output in session_combine.get_outputs()]
print("Inputs shape:", input_shape)
print("Outputs shape:", output_shape)

input_example = {input_name[0]: input_data}

output = session_combine.run(output_name, input_example)[0]
print(output)
print(output*0.018824706 )
print(weight_dnn)
print(output/weight_dnn)

print((output*0.018824706)/weight_dnn)


Inputs name: ['args_0']
Outputs name: ['avg_w']
Inputs shape: [['N', 45]]
Outputs shape: [[None]]
[0.12036134 0.11755123 0.42651352 0.2209754  0.60062635 0.6612763
 2.5380275  1.5624465  0.29646736 0.34631133]
[0.00226577 0.00221287 0.00802899 0.0041598  0.01130661 0.01244833
 0.04777762 0.0294126  0.00558091 0.00651921]
[0.00226577 0.00221286 0.00802899 0.0041598  0.01130661 0.01244833
 0.04777763 0.0294126  0.00558091 0.00651921]
[53.121693 53.12176  53.1217   53.12169  53.121693 53.12168  53.121674
 53.121677 53.121666 53.121662]
[1.0000004  1.0000015  1.0000004  1.0000002  1.0000002  1.
 0.99999994 0.99999994 0.99999976 0.9999997 ]


In [64]:
print(0.00227/0.00221)
print(0.12036134/0.11755123)
print(0.00416/0.00803)
print(0.22097537/0.4265135)
print(0.6006263/0.0113)
print(0.12036134/0.00227)
print(1/53.1)

1.0271493212669682
1.023905407029769
0.5180572851805728
0.5180970121696031
53.15276991150442
53.022616740088104
0.018832391713747645


In [65]:
#read all h5 files in the directory
h5_files = [x for x in os.listdir(main_dir) if x.endswith('.keras')]
print(h5_files)



model = tf.keras.models.load_model(main_dir+'/'+h5_files[0])
pred = model.predict(input_tensor)

print(pred)
model_ratio = tf.keras.models.Model(inputs=model.input, outputs= model.output[:,1]/model.output[:,0])
pred_ratio = model_ratio.predict(input_tensor)
print(pred_ratio)

['model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold10.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold11.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold14.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold15.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold16.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold19.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold2.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold3.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold6.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold7.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold8.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold4.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold9.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold20.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold12.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_fold17.keras', 'model_reweightingTo4b_PASv2p0_sigmaMbbFixed_f

1/1 [==============================] - 0s 107ms/step
[[0.73732585 0.2626741 ]
 [0.94987184 0.05012813]
 [0.5665113  0.4334887 ]
 [0.9496726  0.05032746]
 [0.6109201  0.38907996]
 [0.716273   0.28372696]
 [0.2810895  0.71891046]
 [0.2295987  0.7704013 ]
 [0.8311194  0.16888054]
 [0.99841976 0.00158019]]
1/1 [==============================] - 0s 109ms/step
[3.5625240e-01 5.2773569e-02 7.6518989e-01 5.2994542e-02 6.3687539e-01
 3.9611566e-01 2.5575855e+00 3.3554254e+00 2.0319648e-01 1.5826930e-03]


In [66]:

model1 = tf.keras.models.load_model(main_dir+'/'+h5_files[1])
pred1 = model1.predict(input_tensor)

print(pred1)
model_ratio1 = tf.keras.models.Model(inputs=model1.input, outputs= model1.output[:,1]/model1.output[:,0])
pred_ratio1 = model_ratio1.predict(input_tensor)
print(pred_ratio1)

1/1 [==============================] - 0s 116ms/step
[[0.99755704 0.00244296]
 [0.98007697 0.01992308]
 [0.7394039  0.26059613]
 [0.73482215 0.26517785]
 [0.656055   0.34394506]
 [0.875014   0.12498597]
 [0.3542903  0.6457097 ]
 [0.56431705 0.43568295]
 [0.6095118  0.3904883 ]
 [0.97866416 0.02133583]]
1/1 [==============================] - 0s 123ms/step
[0.00244895 0.02032807 0.35244083 0.3608735  0.52426255 0.14283882
 1.8225441  0.7720535  0.6406575  0.02180097]


In [67]:

model2 = tf.keras.models.load_model(main_dir+'/'+h5_files[2])
pred2 = model2.predict(input_tensor)

print(pred2)
model_ratio2 = tf.keras.models.Model(inputs=model2.input, outputs= model2.output[:,1]/model2.output[:,0])
pred_ratio2 = model_ratio2.predict(input_tensor)
print(pred_ratio2)

1/1 [==============================] - 0s 116ms/step
[[0.9969349  0.00306514]
 [0.9480546  0.05194535]
 [0.78236824 0.21763176]
 [0.97274774 0.02725222]
 [0.86339283 0.13660723]
 [0.5609741  0.43902585]
 [0.40560964 0.5943904 ]
 [0.33525333 0.66474664]
 [0.7483637  0.25163636]
 [0.8556172  0.14438279]]
1/1 [==============================] - 0s 112ms/step
[0.00307457 0.05479152 0.2781705  0.02801571 0.15822141 0.7826134
 1.4654248  1.982819   0.33624876 0.16874695]


In [68]:
print((pred_ratio+pred_ratio1+pred_ratio2)/3)
print(output)
[0.00227, 0.00221, 0.00803, 0.00416, 0.0113]


[0.12059197 0.04263105 0.46526706 0.14729458 0.43978643 0.44052264
 1.9485182  2.0367658  0.39336756 0.06404354]
[0.12036134 0.11755123 0.42651352 0.2209754  0.60062635 0.6612763
 2.5380275  1.5624465  0.29646736 0.34631133]


[0.00227, 0.00221, 0.00803, 0.00416, 0.0113]

In [69]:
1/0.018494148

54.071158076598074